## Report #2

Since we are working strictly with VCF files, we will not need to worry about the FASTQ -> BAM -> VCF conversion through GATK (for now). Instead, we will need to work with software that is capable of manipulating and analyzing VCF files. Therefore, we will be using PLINK2, an open-source genome analysis toolset, to achieve such functionalities. This tool will be used to: filter SNPs, recode data, run PCA to detect outliers, and run PCA to visualize population substructure. The necessary steps for each of these processes, along with some preparatory steps, will be outlined below. 

**<u>Merge VCFs Across Chromosomes<u>**\
The 1000 genomes data set contains separate VCF files for chromosomes 1-22. Since we are interested in genetic variation amongst individuals, merging these files together in order to get a full view of the data is necessary.
    
\
Let's begin by storing all VCF file paths and names into a list file `input.list`:

> ```console
user@dsmlp:~$ ls [filepath] \ 
                | grep .vcf.gz \
                | grep -v ".tbi"\
                | sed -e 's/^/[filepath]/' > input.list
```

Now we will be storing the header of a single VCF file so that we can later concatenate to it:

> ```console
user@dsmlp:~$ zgrep '^#' "$(head -1 input.list)" | gzip > full_chroms.vcf.gz
```

And finally we concatenate all VCF files to a single file `full_chroms.vcf.gz`:
> ```console
user@dsmlp:~$ zgrep -v "^#" $(cat input.list) | gzip >> full_chroms.vcf.gz
```


**<u>Recode Data<u>**\
Since we now have a single VCF file, we must apply a series of variant filters to our data and recode it into an analysis ready format (.bed, .bim, etc.) using PLINK2.
    
This is all conveniently done using this one command:

> ```console
user@dsmlp:~$ plink2 \
                  --vcf [filepath]/full_chroms.vcf.gz \
                  --snps-only \
                  --maf 0.05 \
                  --geno 0.1 \
                  --mind 0.05 \
                  --recode \
                  --allow-extra-chr \
                  --make-bed \
                  --out [filepath]/chromosomes
```

Note that several parameters were passed in to filter the data. Here I will include explanation/justification for these parameters. Please note that any of these parameters can be changed at a later time if necessary as code will be written to make sure of this:

- **--snps-only:** For the purposes of this project, we are focused on genetic variation. Since SNPs are the most common form of genetic variation, we will be filtering the data to include only SNPs.

- **--maf:** Controls the minor allele frequency threshold. It appears to be common practice to set this threshold at 5\% to differentiate between rare and common alleles therefore we will be working with a 5\% threshold.

- **--geno:** Controls the missing call rate threshold. We will keep the default value of 0.1 meaning we want to work with SNPs for which there is less than 10\% missing data.

- **--mind:** Similar to --geno but filters out samples with a certain rate of missing calls. We will be working with a rate of 5\% here.

- **--recode:** Generates a text file set from the VCF file taken in as input (.ped and .map).

- **--allow-extra-chr:** Since we will be filtering across several different chromosomes, this must be allowed for this function to run.

- **--make-bed:** Generates a binary file set from the VCF taken in as input (.bed, .bim, and .fam).

**<u>Run PCA to Detect Outliers<u>** \
We will now be running an initial PCA to detect potential outliers in the data. 

PCA is run on the data generated above with the following command:

> ```console
user@dsmlp:~$ plink2 \
                  --bfile [filepath]/chromosomes \
                  --pca [num_pc] \
                  --allow-extra-chr \
                  --out chrom_pc
```

The **--pca** argument is used to specify the number of principal components needed. We will be working with the top three principal components so that we can visualize the genomic clusters on a three-dimensional plane.

In order to identify outliers, we will be reading in the principal component (eigenvector) file produced from the above command into Python. Samples that are +/- 3 standard deviations from the mean will be deemed outliers and stored in the file `outliers.txt`.

```python 
# Reading in principal component data
pcs = (pd.read_csv('chrom_pc.eigenvec', sep=' ', header=None)
                   .rename(columns={1:'Sample', 2:'PC1', 3:'PC2', 4:'PC3'}))
                   
# Calculating z_scores
z_scores = pcs[['PC1', 'PC2', 'PC3']].apply(lambda x: (x-x.mean())/x.std(), axis=0)

# Finding outlier samples
check_outlier = z_scores.apply(lambda x: abs(x) > 3, axis=0)
outliers = check_outlier[check_outlier.any(axis='columns')].index
outlier_samps = pcs.loc[outliers][[0, 'Sample']]

# Writing outliers to text file
outlier_samps.to_csv('outliers.txt', sep= ' ', header=False, index=False)

```

**<u>Remove Outlier Samples & Rerun PCA<u>**\
We will now be filtering out outliers. If no outliers were found in the previous step, this step will be ignored. Otherwise, the following command will be run to remove samples and rerun PCA.

> ```console
user@dsmlp:~$ plink2 \
                  --bfile [filepath]/chromosomes \
                  --pca [num_pc] \
                  --remove outliers.txt \
                  --allow-extra-chr \
                  --out chrom_pc
```

Note that the only additional argument here **--remove** is given the path to the text file produced in the previous step which contains samples we would like to remove.

**<u>Visualize Population Substructure<u>**\
Now that we have filtered out the outlier samples, we can finally visualize population substructure. Again, we will be reading in the principal component (eigenvector) file produced from the previous step. Additionally, IGSR conveniently provides data on every sample and their associated populations. This will be used to map colors to different super populations when plotting. This data has been slightly modified and stored in a file `sample_pop.csv`.
    
The steps to produce the final plot will be detailed here. Please note that this example is only for chromosomes 3,10, 21, and 22 for the purposes of time. This can be generalized for all chromosome if needed:
    
```python 
# Importing necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px

# Reading in principal component data
pcs = (pd.read_csv('chrom_pc.eigenvec', sep=' ', header=None)
                   .rename(columns={1:'Sample', 2:'PC1', 3:'PC2', 4:'PC3'}))

# Creating sample-superpopulation pair dictionary and 
# mapping to samples in pc data above
samples = pd.read_csv('data/samples/sample_pop.csv')
samples_dict = samples.set_index('Sample').to_dict()['Population']
pcs['Population'] = pcs.loc[:,0].apply(lambda x: samples_dict[x])

# Plotting first three principle components
fig = px.scatter_3d(pcs, x='PC1', y='PC2', z='PC3',color='Population')
fig.update_traces(marker=dict(size=2.5))
fig.update_layout(width=800, 
                  margin=dict(r=20, l=50, b=70, t=30), 
                  legend={'x':.75, 'y':.85, 'itemsizing':'constant'},
                  title={'text': "Clustering Chromosomes 3,10,21,22", 
                  'y':0.9,'x':0.53},
                  font=dict(size=15))
    
fig.show()
```

<img src="../data/raw/cluster.png" width="480"> 

**<u>Final Note<u>**\
The general workflow of this process is highlighted above. However, a pipeline will be written to generalize and organize this all together. Therefore, there will likely be some slight modifications to the steps above in terms of implementation.
    